In [ ]:
!pip install ultralytics kaggle tqdm albumentations

In [ ]:
import os
import shutil

kaggle_json_src = "/kaggle/input/kaggle.json"
kaggle_json_dest = "/root/.kaggle/kaggle.json"

if os.path.exists(kaggle_json_src):
    os.makedirs("/root/.kaggle/", exist_ok=True)
    shutil.copyfile(kaggle_json_src, kaggle_json_dest)
    os.chmod(kaggle_json_dest, 0o600)
else:
    print("kaggle.json not found. Please add it to the notebook inputs.")

In [ ]:
!git clone https://github.com/n24q02m/VehicleDetection.git
%cd VehicleDetection

In [ ]:
import os
import sys

repo_path = "/kaggle/working/VehicleDetection"
if repo_path not in sys.path:
    sys.path.append(repo_path)

In [ ]:
from src.training.train import main as train_main

# Hoặc gọi hàm train_main()
train_main()